# Соревнование по Machine Learning
# Задача по рекомендательным системам

В данном задании вам предстоит построить рекомендательную систему для образовательной онлайн платформы **Отус**. А именно, вам необходимо обучить модель, которая для каждого пользователя (имя пользователя закодировано через `user_id`) будет рекомендовать ему 3 курса (`course_id`) из тех, которые он еще не слушал. При этом модель должна уметь работать как со старыми пользователями, так и с новыми, которые еще не приобрели ни одного курса.

## Данные
Для построения модели вам будут доступны различные данные о взятых пользователями курсах, выполненных домашних заданиях и тестах.

Архив с данными содержит следующие файлы.

### Обучающие данные

- **user_course_train.csv** - таблица пользователь-курс, **основная таблица** с информацией о взятых пользователями курсах 


- **assessment_train.csv** - информация о том какие тесты и с каким результатом прошел пользователь

- **course.csv** - информация обо всех курсах 

- **lessons_homework_train.csv** - информация о всех сданных домашних работах

- **sample_submission.csv** - пример сабмита


###  Тестовые данные

- **test_ids.txt** - список пользователей (`user_id`) для которых необходимо сделать предсказания

In [1]:
import pandas as pd

**user_course_train.csv**


Таблица пользователь-курс, **основная таблица** с информацией о взятых пользователями курсах 

К наиболее важным колонкам относятся:
- user_id - уникальный идентификатор пользователя
- item_id - уникальный идетификатор курса

In [2]:
train = pd.read_csv('../input/otus-rec-sys/user_course_train.csv', index_col = 0)
train.head()

,user_id,course_id,paid,month_every_discount,full_paid,money_used,month_used,id,group_id,hidden_description,...,max_residual_discount,residual_discount,residual_percent_discount,show_project_work,show_sign,show_stamp,subscription_id,certificate_company_title,certificate_company_title_latin,show_company_info
0,4,1,0,0,0,0,0,4,NaN,NaN,...,0.30,0,0.00,0,1.0,1,NaN,NaN,NaN,0
17,81,1,7000,3500,0,0,0,81,57,NaN,...,0.30,0,0.00,0,1.0,1,NaN,NaN,NaN,0
19,83,1,0,3500,0,0,0,83,NaN,NaN,...,0.30,0,0.00,0,1.0,1,NaN,NaN,NaN,0
43,120,1,36000,1000,1,0,0,120,191,NaN,...,0.30,0,0.00,1,1.0,1,NaN,NaN,NaN,0
46,123,1,0,3500,0,0,0,123,NaN,NaN,...,0.30,0,0.00,0,1.0,1,NaN,NaN,NaN,0


**test**

Тестовый набор `user_id` для которого необходимо сделать предсказания.

In [3]:
test = [int(x) for x in open('../input/otus-rec-sys/test_ids.txt').readlines()]
print(len(test))
test[:5]

3910


[163840, 122882, 81926, 8, 73737]

**course**

Таблица содержит расшифровку `course_id` и подробную информацию о курсах.

In [4]:
courses = pd.read_csv('../input/otus-rec-sys/course.csv', index_col = 0, sep = ';')
courses.head(2)

,title,salary,enabled,description,icon,assessment_duration,shortname,assessment_min,short_description,program,...,ga_goal_request_without_test_send,ya_goal_request_without_test_not_send,ya_goal_request_without_test_send,text_assessment_before_test_with_test_description,text_assessment_before_test_with_test_header,text_assessment_before_test_without_test_description,text_assessment_before_test_without_test_header,dont_show_in_schedule,tilda_html,tilda_page_id
id,,,,,,,,,,,,,,,,,,,,,
1,Java Developer. Professional,160000.0,1.0,<p>Вы получите <strong>практику</strong> решен...,d7/19/d71915b397a94461827988c04e4e9950.png,30.0,Java,8.0,На курсе изучаются особенности языка и платфор...,<ol>\n<li> Введение. Обзор и задачи курса. ...,...,NaN,NaN,NaN,NaN,<p>Для начала тестирования необходимо зарегист...,NaN,"<p>Для того чтобы оставить заявку, нужно зарег...",0,NaN,NaN
2,Разработчик Android (deprecated),140000.0,0.0,Длинное-длинное-длинное описание профессии And...,NaN,30.0,Android,2.0,Короткое описание профессии Android,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN


**assessment_train**

Таблица содержит информацию о том, какие тесты и с каким результатом прошел пользователь.

In [5]:
pd.read_csv('../input/otus-rec-sys/assessment_train.csv', index_col = 0).head(2)

,id,user_id,course_id,start,finish,result,status,group_id,last_update,partner_campaign_id,finish_for_dod,mark,user_session_id,utm,lead_id,visible_mark,parent_id,force_amnesty
5,64,81,1,2016-05-02 09:14:00,05.02.2016 09:44,9,OK,1,0,NaN,05.02.2016 09:44,B,NaN,NaN,NaN,D,NaN,0
11,86,120,1,2016-05-02 16:04:00,05.02.2016 16:33,9,OK,1,0,NaN,24.05.2018 20:23,B,NaN,NaN,NaN,D,NaN,0


**lessons_homework_train**

Таблица содержит информацию о сданных домашних работах.

In [6]:
pd.read_csv('../input/otus-rec-sys/lessons_homework_train.csv', index_col = 0).head(2)

,id,points,comment,homework_id,user_id,assignee_id,status,status_update_time,last_message_time,first_attempt_time,is_was_revoked,project_theme,project_theme_status
0,1,0,NaN,20,5950,16,new,2018-02-27 04:30:00,NaN,NaN,0,NaN,new
4,5,8,NaN,1,800,22,accept,2018-02-27 04:30:00,NaN,NaN,0,NaN,new


# Baseline

В качестве baseline мы предлагаем MostPopular алгоритм, который рекомендует пользователю самые популярные курсы из тех, что он еще не брал. Для данного алгоритма нам понадобится всего две колонки `user_id`,`course_id` из таблицы `user_course`.


In [7]:
X = train[['user_id','course_id']].copy().drop_duplicates()
X.head()

,user_id,course_id
0,4,1
17,81,1
19,83,1
43,120,1
46,123,1


Для отладки алгоритма разобьем данные на train и test. В baseline решении используется самая простая разбивка (на то это и baseline), возможно, вы захотите сделать что-то более умное, например, учитывающие id юзера.

In [8]:
from sklearn.model_selection import train_test_split

X_train, y_val = train_test_split(X, random_state= 1, test_size = 0.3)
print(X_train.shape, y_val.shape)
val_idx = list(sorted(set(y_val.user_id)))

(10272, 2) (4403, 2)


### Most Popular Alogoritm
На этапе обучения вычисляет наиболее популярные курсы в обучающих данных, на этапе предсказания - выбирает top popular, из тех которые пользователь еще не брал.

In [9]:
from sklearn.base import BaseEstimator
from typing import List

class MostPopular(BaseEstimator):
    def fit(self, train:pd.DataFrame):
        self.most_popular = list(train['course_id'].value_counts().index)
        self.users_items = train[['user_id', 'course_id']]
        return self
    
    def predict_items(self, user_id:str, n:int = 3):
        taken_courses = set(self.users_items[self.users_items['user_id'] == user_id]['course_id'])
        return [x for x in self.most_popular if not x in taken_courses][:n]
    
    def predict(self, user_ids:List[str], n = 3):
        y = [self.predict_items(user_id, n = n) for user_id in user_ids]
        return y

In [10]:
# Обучаем модель
estimator = MostPopular().fit(X_train)

# Метрика 
В качестве метрики качества в соревновании используется **MeanAveragePrecision@3**:

$$ AveragePrecision@3(user) = \frac{1}{3}\sum_{i = 1}^3y_i \cdot presicion@i(user)$$

$y_i \in {0,1}$ - бинарная метка $i$-го предсказания (верно предсказал/неверно);

$presicion@i(user)$ - доля верно предсказанных среди первых $i$ курсов. 

Итоговая метрика считается усреднением по всем юзерам:

$$MeanAveragePrecision@3 = \frac{1}{|U|}\sum_{user \in U} AveragePrecision@3(user) $$

Подробное описание метрики можно прочитать тут: https://habr.com/ru/company/econtenta/blog/303458/

При оценке используется решения используется реализация метрики из библиотеки `ml_metrics`: https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py

In [11]:
#!pip install ml_metrics

In [12]:
from ml_metrics import mapk

In [13]:
val_preds = estimator.predict(val_idx)
y_val_true = [list(set(y_val[y_val.user_id == idx].course_id)) for idx in val_idx]
val_preds[:3], y_val_true[:3]

([[7, 1, 15], [7, 15, 3], [7, 15, 34]], [[1], [51], [51]])

Помотрим на названия рекомендаций:

In [14]:
id2course = {}
for i, row in courses.iterrows():
    id2course[i] = row.title
    
print([id2course[i] for i in val_preds[0]])

['DevOps практики и инструменты', 'Java Developer. Professional', 'Administrator Linux. Professional']


In [15]:
# Вычислим MAP@3
print('MAP@3 =', round(mapk(
    y_val_true, 
    val_preds, 
    k=3
),3))

MAP@3 = 0.109


# Submission

В качестве ответа вам необходимо предоставить `csv` файл с двумя колонками:

* **Id** (type int64) с id предсказаний;
    
* **Predicted** (type string), в которой для каждого юзера должно быть записано топ3 рекомендованных курса. **Курсы должны быть записаны в виде строки через пробел** (Пример: `7 1 15`).
    

<center> 
    <table>
        <tr>
            <th>  </th>
            <th> Id </th>
            <th> Predicted </th>
        </tr> 
        <tr><td> 0 </td><td> 163840 </td><td> 7 1 15 </td></tr>
        <tr><td> 1 </td><td> 122882 </td><td> 7 1 15 </td></tr>
        <tr><td> 2 </td><td> 81926 </td><td> 7 1 15 </td></tr>
      
</center>
 

In [16]:
estimator = MostPopular().fit(train)
test_preds = estimator.predict(test, n = 3)

In [17]:
submission = pd.DataFrame([test, test_preds], index = ['Id', 'Predicted']).T
submission['Predicted'] = submission['Predicted'].map(list)
submission['Predicted'] = submission['Predicted'].apply(lambda x: " ".join(map(str, x)))
submission.head(3)

,Id,Predicted
0,163840,7 1 15
1,122882,7 1 15
2,81926,7 1 15


In [18]:
submission.to_csv('sample_submission.csv', index=None)